In [1]:
from ultralytics import YOLO
import optuna
import yaml
import torch

WARNING ⚠️ user config directory '/root/.config/Ultralytics' is not writeable, defaulting to '/tmp' or CWD.Alternatively you can define a YOLO_CONFIG_DIR environment variable for this path.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/tmp/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
print(torch.cuda.is_available())
print(torch.version.cuda)
torch.cuda.empty_cache()

True
12.1


In [3]:
# === Optuna ===

# === SETTINGS ===
MODEL_ARCH = "yolov8m.yaml"
DATASET_PATH = "dataset/dataset_night.yaml"
IMG_SIZE = 640
EPOCHS = 10
DEVICE = 0

def objective(trial):
    # HYPERPARAMETERS
    hyp = {
        'lr0': trial.suggest_float('lr0', 1e-4, 1e-1, log=True),
        'momentum': trial.suggest_float('momentum', 0.6, 0.98),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True),
        'hsv_h': trial.suggest_float('hsv_h', 0.0, 0.1),
        'hsv_s': trial.suggest_float('hsv_s', 0.5, 1),
        'hsv_v': trial.suggest_float('hsv_v', 0.2, 1),
        'translate': trial.suggest_float('translate', 0.0, 0.2),
        'shear': trial.suggest_float('shear', 0.0, 2.0),
        'fliplr': trial.suggest_float('fliplr', 0.0, 0.5),
        'mosaic': trial.suggest_float('mosaic', 0.0, 1.0),
        'mixup': trial.suggest_float('mixup', 0.0, 0.5),
    }

    model = YOLO(MODEL_ARCH)

    model.train(
        data=DATASET_PATH,
        imgsz=IMG_SIZE,
        epochs=EPOCHS,
        batch=32,
        workers=2,
        device=DEVICE,
        optimizer='SGD',
        verbose=False,
        patience=2,
        lr0=hyp['lr0'],
        momentum=hyp['momentum'],
        weight_decay=hyp['weight_decay'],
        hsv_h=hyp['hsv_h'],
        hsv_s=hyp['hsv_s'],
        hsv_v=hyp['hsv_v'],
        translate=hyp['translate'],
        scale=1,
        shear=hyp['shear'],
        fliplr=hyp['fliplr'],
        mosaic=hyp['mosaic'],
        mixup=hyp['mixup'],
        project="optuna_yolo_tune",
        name=f"trial_{trial.number}",
    )

    # VALIDATE
    try:
        metrics = model.val()
        map50 = metrics.box.map50
    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        return 0.0

    return map50

# === EXECUTE OPTUNA ===
study = optuna.create_study(
    direction="maximize",
    study_name="yolov8_tune",
    storage="sqlite:///optuna_yolov8.db",
    load_if_exists=True
)
study.optimize(objective, n_trials=10)

print("\n✅ Best hyperparameteres:")
for key, value in study.best_params.items():
    print(f"{key}: {value}")

# Zapisz do pliku YAML
with open("cfg/best_hyp.yaml", "w") as f:
    yaml.dump(study.best_params, f)

print("\n📄Saved best hyperparameters to best_hyp.yaml")

[I 2025-05-24 13:39:17,830] Using an existing study with name 'yolov8_tune' instead of creating a new one.


New https://pypi.org/project/ultralytics/8.3.144 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.143 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24107MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/dataset_night.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.30415414448221206, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.045355002195497485, hsv_s=0.5663595957972734, hsv_v=0.7511002830170874, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01153283383476797, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.05511206826299392, mode=train, model=yolov8m.yaml,

100%|██████████| 755k/755k [00:00<00:00, 4.88MB/s]

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     


  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384, 576, 3, 2]              
  8                  -1  2   3985920  ultralytics.nn.modules.block.C2f             [576,

...

                   all      10779      39629      0.373      0.262      0.249      0.119
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to optuna_yolo_tune/trial_14
Ultralytics 8.3.143 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24107MiB)
YOLOv8m summary (fused): 92 layers, 25,843,234 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 1.0±0.9 ms, read: 31.4±15.5 MB/s, size: 70.9 KB)


val: Scanning dataset/night/test/nightSequence1/frames.cache... 10779 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10779/10779 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 337/337 [00:40<00:00,  8.30it/s]


                   all      10779      39629      0.374      0.262       0.25      0.119
Speed: 0.1ms preprocess, 2.1ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to optuna_yolo_tune/trial_142


[I 2025-05-24 15:29:24,259] Trial 14 finished with value: 0.2496764798879525 and parameters: {'lr0': 0.013242975743539274, 'momentum': 0.7050992867880947, 'weight_decay': 0.0034461557389827806, 'hsv_h': 0.07353790506176106, 'hsv_s': 0.7623518816341652, 'hsv_v': 0.7230836652476453, 'translate': 0.11592908122097861, 'shear': 1.6290144979462986, 'fliplr': 0.21427978862326574, 'mosaic': 0.3650945389377531, 'mixup': 0.33249074810397644}. Best is trial 7 with value: 0.3426744628219599.



✅ Best hyperparameteres:
lr0: 0.012206857255596018
momentum: 0.9406589735310404
weight_decay: 6.795431495220002e-05
hsv_h: 0.03234048558652525
hsv_s: 0.5209106238368573
hsv_v: 0.5806064113318283
translate: 0.026595217915804395
shear: 0.3471662587789368
fliplr: 0.333531443624219
mosaic: 0.7724714127761845
mixup: 0.055993553020083076

📄Saved best hyperparameters to best_hyp.yaml
